In [1]:
import numpy as np
import sys
import os
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir("/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/lung/All+SMI+Flat+data")

In [3]:
!ls
!pwd

1.csv			 Lung12  Lung5_Rep1  Lung5_Rep3  Lung9_Rep1  vis   vis2
cosMX_showcase_meta.csv  Lung13  Lung5_Rep2  Lung6	 Lung9_Rep2  vis1  vis3
/home/luqiaolin/projects/Benchmarking_paper_code/cosmx_spot_data/pretrain/lung/All+SMI+Flat+data


# Transcript Data

In [4]:
gene_expression = pd.read_csv('./Lung5_Rep1/Lung5_Rep1-Flat_files_and_images/Lung5_Rep1_exprMat_file.csv')
cell_boundary = pd.read_csv('./Lung5_Rep1/Lung5_Rep1-Flat_files_and_images/Lung5_Rep1_metadata_file.csv')
fov_position = pd.read_csv('./Lung5_Rep1/Lung5_Rep1-Flat_files_and_images/Lung5_Rep1_fov_positions_file.csv')


In [5]:
x_coordinate = [x / 10000 for x in list(fov_position["x_global_px"])]
y_coordinate = [y / 10000 for y in list(fov_position["y_global_px"])]


# Benchmark Generation

In [6]:
"""
CosMx: 
1. All FOVs are the same dimension, 5472 x 3648 pixels
2. multiply the pixel value by 0.18 um per pixel
3. FOV area: 5472 x 3648 pixels -> 984.96um x 656.64um = 646,764.134 um2 


New Benchamrk from CosMx
1. length: 5472 pixels, width: 3648 pixels
2. simulated spot: 
    length: 5472 pixels / 5 = 1094.4 pixel = 196.992 um
    width: 3648 pixels / 4 = 912 pixel = 164.16 um
    one spot area: 196.992 um * 164.16 um = 32338.2067 um2
3. In total: 20 spots / FOV

spot_fov_cellId_mapping.csv (spot_id, fov, cell_Id)

spot_gene_expression.csv (spot_id, gene1, gene2, gene3...)

"""

'\nCosMx: \n1. All FOVs are the same dimension, 5472 x 3648 pixels\n2. multiply the pixel value by 0.18 um per pixel\n3. FOV area: 5472 x 3648 pixels -> 984.96um x 656.64um = 646,764.134 um2 \n\n\nNew Benchamrk from CosMx\n1. length: 5472 pixels, width: 3648 pixels\n2. simulated spot: \n    length: 5472 pixels / 5 = 1094.4 pixel = 196.992 um\n    width: 3648 pixels / 4 = 912 pixel = 164.16 um\n    one spot area: 196.992 um * 164.16 um = 32338.2067 um2\n3. In total: 20 spots / FOV\n\nspot_fov_cellId_mapping.csv (spot_id, fov, cell_Id)\n\nspot_gene_expression.csv (spot_id, gene1, gene2, gene3...)\n\n'

## 1. spot_fov_cellId_mapping.csv

In [7]:
def get_spot_fov_cellId_mapping(data_result, cell_boundary_fov_11):
    new_col_val = cell_boundary_fov_11.shape[0]* [0]
    cell_boundary_fov_11.insert(loc=0, column='spot_id', value=new_col_val)

    for i in range(cell_boundary_fov_11.shape[0]):
        one_row_sample = cell_boundary_fov_11.iloc[i]

        center_x = one_row_sample["CenterX_local_px"]
        center_y = one_row_sample["CenterY_local_px"]

        # Calculate spot_id based on CenterX_local_px and CenterY_local_px values
        spot_id = 1 + 4 * int((center_x - 1e-6) / 1094.4) + int((center_y - 1e-6) / 912)

        # Use spot_id as needed in your code
        # print(spot_id)
        data_result = data_result.append({'spot_id' : spot_id, 'fov' : one_row_sample["fov"], 'cell_ID' : one_row_sample["cell_ID"]}, ignore_index = True)
    return data_result
    

In [8]:
data_final_result = pd.DataFrame(columns = ['spot_id', 'fov', 'cell_ID'])
fov_ids_lst = cell_boundary['fov'].unique()
print("fov_ids_lst:", fov_ids_lst)

for fov_id in tqdm(fov_ids_lst):
    # print("fov_id:", fov_id)
    cell_boundary_fov =  cell_boundary[(cell_boundary['fov']==fov_ids_lst[fov_id-1])]
    # print("cell_boundary_fov_without_spot_id:", cell_boundary_fov.shape)
    data_final_result = get_spot_fov_cellId_mapping(data_final_result, cell_boundary_fov)


fov_ids_lst: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]


  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [01:29<00:00,  2.99s/it]


In [9]:
# data_final_result.to_csv('./Lung5_Rep1/Lung5_Rep1-Flat_files_and_images/new/spot_fov_cellId_mapping.csv')

## 2. spot_gene_expression.csv

In [10]:
# data_final_result = pd.read_csv('./Lung5_Rep1/Lung5_Rep1-Flat_files_and_images/new/spot_fov_cellId_mapping.csv')
spot_celld_mapping = data_final_result

In [11]:
gene_expression

,fov,cell_ID,AATK,ABL1,ABL2,ACE,ACE2,ACKR1,ACKR3,ACKR4,...,NegPrb14,NegPrb15,NegPrb16,NegPrb17,NegPrb18,NegPrb19,NegPrb20,NegPrb21,NegPrb22,NegPrb23
0,1,0,15,38,16,29,13,51,9,17,...,18,7,9,11,14,12,31,11,13,13
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100317,30,4756,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100318,30,4757,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100319,30,4758,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100320,30,4759,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
names = ['fov'] + ['spot-id=' + str(i) for i in range(1,21)]
fov_dic = {}
for i in names:
    fov_dic[i] = 0

In [13]:
fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = [i for i in range(1,21)]


names = ['fov'] + ['spot-id=' + str(i) for i in range(1,21)]
fov_spot_cells_stats = pd.DataFrame(columns = names)


for fov_id in fov_ids_lst:
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    
    fov_dic_sample = fov_dic
    fov_dic_sample["fov"] = fov_id
    
    for i in spot_id_lst:
        spot_id_data = fov_data[(fov_data['spot_id']==i)]
        spot_id_num = "spot-id=" + str(i)
        fov_dic_sample[spot_id_num] = spot_id_data.shape[0]
    fov_spot_cells_stats = fov_spot_cells_stats.append(fov_dic_sample, ignore_index = True)



In [14]:
fov_spot_cells_stats

,fov,spot-id=1,spot-id=2,spot-id=3,spot-id=4,spot-id=5,spot-id=6,spot-id=7,spot-id=8,spot-id=9,...,spot-id=11,spot-id=12,spot-id=13,spot-id=14,spot-id=15,spot-id=16,spot-id=17,spot-id=18,spot-id=19,spot-id=20
0,1,155,119,113,130,88,108,128,178,128,...,102,110,89,87,131,131,101,130,140,158
1,2,141,233,148,98,371,207,119,124,237,...,140,147,272,259,146,141,269,169,187,188
2,3,188,175,233,117,150,75,112,169,111,...,145,222,142,119,89,136,171,142,128,129
3,4,179,134,131,170,165,129,227,204,173,...,242,258,149,176,131,142,223,178,206,186
4,5,188,209,291,241,174,157,279,226,137,...,130,191,151,152,164,182,102,124,188,163
5,6,288,247,276,254,184,306,318,264,247,...,204,266,221,236,291,346,207,228,349,255
6,7,225,285,220,165,300,250,108,139,141,...,158,182,173,223,186,187,172,150,111,150
7,8,183,92,170,161,89,195,125,209,149,...,223,97,173,126,168,175,161,172,167,155
8,9,222,168,170,140,143,210,175,151,225,...,166,122,141,167,198,140,146,138,165,148
9,10,170,132,133,149,157,149,127,117,229,...,173,119,195,190,156,149,117,144,147,184


In [15]:
spot_gene_expression = ["fov", "spot_id"]
genes_name_lst = list(gene_expression.columns)[2:]
spot_gene_expression = spot_gene_expression + genes_name_lst
spot_gene_expression = pd.DataFrame(columns = spot_gene_expression)


In [16]:
def get_spot_gene_expression(spot_cell_mapping, fov_expression, spot_id):
    genes_lst = list(fov_expression.columns)[2:]
    
    cell_id_lst = spot_cell_mapping[(spot_cell_mapping['spot_id']==spot_id)]["cell_ID"].tolist()
    
    cell_gene_expression_total = len(genes_lst)*[0]
    for cell_id in cell_id_lst:
        cell_gene_expression = fov_expression[(fov_expression['cell_ID'] == cell_id)]
        cell_gene_expression = cell_gene_expression.values.tolist()[0][2:]
        cell_gene_expression_total = np.sum([cell_gene_expression_total, cell_gene_expression], axis=0).tolist()
    
    return cell_gene_expression_total
    

In [17]:
fov_ids_lst = cell_boundary['fov'].unique()
spot_id_lst = [i for i in range(1,21)]

for fov_id in tqdm(fov_ids_lst):
    fov_data = data_final_result[(data_final_result['fov']==fov_id)]
    fov_gene_expression = gene_expression[(gene_expression['fov'] == fov_id)]
    
    for spot_id in spot_id_lst:
        to_append = [fov_id, spot_id]
        spot_gene_express = get_spot_gene_expression(fov_data, fov_gene_expression, spot_id)
        to_append = to_append + spot_gene_express
        a_series = pd.Series(to_append, index = spot_gene_expression.columns)
        spot_gene_expression = spot_gene_expression.append(a_series, ignore_index=True)    
  

100%|██████████| 30/30 [00:40<00:00,  1.34s/it]


In [18]:
# spot_gene_expression.to_csv('./Lung5_Rep1/Lung5_Rep1-Flat_files_and_images/new/spot_gene_expression.csv')

In [19]:
CosMx_cell_type = pd.read_csv('./cosMX_showcase_meta.csv')
sample_cell_count_dic = {}
sample_fov_count_dic = {}
for i in range(1, 9):
    sample_cell_count_dic[i] = 0
    sample_fov_count_dic[i] = 0
# print("cell:", sample_cell_count_dic)
# print("fov:", sample_fov_count_dic)

sample_1_index_lst = []

cell_id_lst = CosMx_cell_type["cell_ID"].tolist()
# print("len of cell_id_lst:", len(cell_id_lst))

fov_lst = []

index = 0
for i in range(len(cell_id_lst)):
    sample_id = int(cell_id_lst[i].split("_")[1])
    fov_id = int(cell_id_lst[i].split("_")[2])
    cell_id = int(cell_id_lst[i].split("_")[3])

    if sample_id == 1:
        sample_1_index_lst.append(index)
        
    sample_cell_count_dic[sample_id] =  sample_cell_count_dic[sample_id] + 1
    if sample_id == 1:
        if fov_id not in fov_lst:
            fov_lst.append(fov_id)
    index += 1

print(sample_cell_count_dic)
print(fov_lst)
print(len(sample_1_index_lst))


{1: 98002, 2: 105800, 3: 97809, 4: 89975, 5: 87606, 6: 139504, 7: 71304, 8: 81236}
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
98002


## 3. Ground Truth

In [20]:
CosMx_cell_type_sample_1 = CosMx_cell_type.iloc[sample_1_index_lst]
CosMx_cell_type_sample_1.reset_index(drop=True, inplace=True)
CosMx_cell_type_sample_1

,Unnamed: 0,cell_ID,fov,Area,AspectRatio,Width,Height,Mean.CD298,Max.CD298,Mean.G,...,Panel,Diversity,totalcounts,log10totalcounts,background,remove_flagged_cells,patient,cell_type,niche,prop_tumor_in_100_neighbors
0,1,c_1_1_1,1,1259,1.34,47,35,3473,7354,715,...,980p,19,23,1.361728,0.004956,False,Lung5,mDC,plasmablast-enriched stroma,0.09
1,2,c_1_1_2,1,3723,1.45,87,60,3895,13832,18374,...,980p,23,26,1.414973,0.005603,False,Lung5,endothelial,stroma,0.00
2,3,c_1_1_3,1,2010,1.62,68,42,2892,6048,3265,...,980p,51,74,1.869232,0.015946,False,Lung5,endothelial,stroma,0.01
3,4,c_1_1_4,1,3358,0.47,48,102,6189,16091,485,...,980p,48,60,1.778151,0.012929,False,Lung5,pDC,stroma,0.00
4,5,c_1_1_5,1,1213,1.00,38,38,8138,19281,549,...,980p,39,52,1.716003,0.011205,False,Lung5,macrophage,stroma,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97997,97998,c_1_30_4750,30,1614,2.06,64,31,889,2718,178,...,980p,31,47,1.672098,0.010128,False,Lung5,mDC,stroma,0.00
97998,97999,c_1_30_4751,30,1346,2.83,68,24,577,1840,167,...,980p,53,97,1.986772,0.020902,False,Lung5,fibroblast,stroma,0.00
97999,98000,c_1_30_4752,30,1336,1.86,54,29,309,942,77,...,980p,50,82,1.913814,0.017670,False,Lung5,mDC,lymphoid structure,0.00
98000,98001,c_1_30_4753,30,803,2.58,49,19,834,3552,142,...,980p,16,25,1.397940,0.005387,False,Lung5,B-cell,lymphoid structure,0.00


In [21]:

for i in CosMx_cell_type_sample_1.index:
    CosMx_cell_type_sample_1.at[i,'cell_ID']= int(CosMx_cell_type_sample_1.iloc[i]["cell_ID"].split("_")[3])

CosMx_cell_type_sample_1

,Unnamed: 0,cell_ID,fov,Area,AspectRatio,Width,Height,Mean.CD298,Max.CD298,Mean.G,...,Panel,Diversity,totalcounts,log10totalcounts,background,remove_flagged_cells,patient,cell_type,niche,prop_tumor_in_100_neighbors
0,1,1,1,1259,1.34,47,35,3473,7354,715,...,980p,19,23,1.361728,0.004956,False,Lung5,mDC,plasmablast-enriched stroma,0.09
1,2,2,1,3723,1.45,87,60,3895,13832,18374,...,980p,23,26,1.414973,0.005603,False,Lung5,endothelial,stroma,0.00
2,3,3,1,2010,1.62,68,42,2892,6048,3265,...,980p,51,74,1.869232,0.015946,False,Lung5,endothelial,stroma,0.01
3,4,4,1,3358,0.47,48,102,6189,16091,485,...,980p,48,60,1.778151,0.012929,False,Lung5,pDC,stroma,0.00
4,5,5,1,1213,1.00,38,38,8138,19281,549,...,980p,39,52,1.716003,0.011205,False,Lung5,macrophage,stroma,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97997,97998,4750,30,1614,2.06,64,31,889,2718,178,...,980p,31,47,1.672098,0.010128,False,Lung5,mDC,stroma,0.00
97998,97999,4751,30,1346,2.83,68,24,577,1840,167,...,980p,53,97,1.986772,0.020902,False,Lung5,fibroblast,stroma,0.00
97999,98000,4752,30,1336,1.86,54,29,309,942,77,...,980p,50,82,1.913814,0.017670,False,Lung5,mDC,lymphoid structure,0.00
98000,98001,4753,30,803,2.58,49,19,834,3552,142,...,980p,16,25,1.397940,0.005387,False,Lung5,B-cell,lymphoid structure,0.00


In [22]:
sample_1_dic = {}
for key in CosMx_cell_type_sample_1["cell_type"].tolist():
    if key not in sample_1_dic:
        sample_1_dic[key] = 1
    else:
        sample_1_dic[key] = sample_1_dic[key] + 1
# sample_1_dic


In [23]:
cell_type_lst = list(CosMx_cell_type_sample_1['cell_type'].unique())
print(len(cell_type_lst))
cell_type_lst

22


['mDC',
 'endothelial',
 'pDC',
 'macrophage',
 'plasmablast',
 'epithelial',
 'neutrophil',
 'fibroblast',
 'NK',
 'T CD4 naive',
 'B-cell',
 'tumor 5',
 'T CD8 naive',
 'mast',
 'T CD4 memory',
 'T CD8 memory',
 'Treg',
 'monocyte',
 'tumor 12',
 'tumor 6',
 'tumor 13',
 'tumor 9']

In [24]:
CosMx_cell_type_sample_1

,Unnamed: 0,cell_ID,fov,Area,AspectRatio,Width,Height,Mean.CD298,Max.CD298,Mean.G,...,Panel,Diversity,totalcounts,log10totalcounts,background,remove_flagged_cells,patient,cell_type,niche,prop_tumor_in_100_neighbors
0,1,1,1,1259,1.34,47,35,3473,7354,715,...,980p,19,23,1.361728,0.004956,False,Lung5,mDC,plasmablast-enriched stroma,0.09
1,2,2,1,3723,1.45,87,60,3895,13832,18374,...,980p,23,26,1.414973,0.005603,False,Lung5,endothelial,stroma,0.00
2,3,3,1,2010,1.62,68,42,2892,6048,3265,...,980p,51,74,1.869232,0.015946,False,Lung5,endothelial,stroma,0.01
3,4,4,1,3358,0.47,48,102,6189,16091,485,...,980p,48,60,1.778151,0.012929,False,Lung5,pDC,stroma,0.00
4,5,5,1,1213,1.00,38,38,8138,19281,549,...,980p,39,52,1.716003,0.011205,False,Lung5,macrophage,stroma,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97997,97998,4750,30,1614,2.06,64,31,889,2718,178,...,980p,31,47,1.672098,0.010128,False,Lung5,mDC,stroma,0.00
97998,97999,4751,30,1346,2.83,68,24,577,1840,167,...,980p,53,97,1.986772,0.020902,False,Lung5,fibroblast,stroma,0.00
97999,98000,4752,30,1336,1.86,54,29,309,942,77,...,980p,50,82,1.913814,0.017670,False,Lung5,mDC,lymphoid structure,0.00
98000,98001,4753,30,803,2.58,49,19,834,3552,142,...,980p,16,25,1.397940,0.005387,False,Lung5,B-cell,lymphoid structure,0.00


In [25]:
cell_id_lst = CosMx_cell_type_sample_1["cell_ID"].tolist()
len(cell_id_lst)

98002

In [26]:
def get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type, cell_type_dic):
    existig_cell_ids_with_celltype = CosMx_cell_type['cell_ID'].unique()
    
    for cell_id in one_spot_cell_lst:
        if cell_id in existig_cell_ids_with_celltype:
            one_cell_sample = CosMx_cell_type[(CosMx_cell_type['cell_ID']==cell_id)]
            cell_type = one_cell_sample["cell_type"].values[0]
            cell_type_dic[cell_type] = cell_type_dic[cell_type] + 1
        else:
            print("cell id not found in groud truth!!!")
        
    return cell_type_dic


In [27]:
fov_ids_lst = spot_celld_mapping['fov'].unique()
spot_id_lst = [i for i in range(1,21)]

column_name_lst = ['fov', 'spot_id'] + cell_type_lst
ground_truth_table = pd.DataFrame(columns = column_name_lst)

for fov_id in fov_ids_lst:
    fov_data = spot_celld_mapping[(spot_celld_mapping['fov']==fov_id)]

    for spot_id in spot_id_lst:
        sample_dic = {}
        for i in column_name_lst:
            sample_dic[i] = 0
        
        spot_id_data = fov_data[(fov_data['spot_id']==spot_id)]
        one_spot_cell_lst = list(spot_id_data['cell_ID'].unique()) # all cell ids for one specific spot
        
        CosMx_cell_type_sample_fov = CosMx_cell_type_sample_1[(CosMx_cell_type_sample_1['fov']==fov_id)]
        sample_dic = get_spot_cell_type_dic(one_spot_cell_lst, CosMx_cell_type_sample_fov, sample_dic)
        
        sample_dic["fov"] = fov_id
        sample_dic["spot_id"] = spot_id
        ground_truth_table = ground_truth_table.append(sample_dic, ignore_index = True)




cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud truth!!!
cell id not found in groud t

In [28]:
ground_truth_table.rename(columns = {'tumor 5':'tumor'}, inplace = True)
del ground_truth_table['tumor 12']
del ground_truth_table['tumor 13']
del ground_truth_table['tumor 6']
del ground_truth_table['tumor 9']

In [29]:
ground_truth_table
# ground_truth_table.to_csv('./Lung5_Rep1/Lung5_Rep1-Flat_files_and_images/new/ground_truth.csv')

,fov,spot_id,mDC,endothelial,pDC,macrophage,plasmablast,epithelial,neutrophil,fibroblast,NK,T CD4 naive,B-cell,tumor,T CD8 naive,mast,T CD4 memory,T CD8 memory,Treg,monocyte
0,1.0,1.0,3.0,40.0,5.0,5.0,12.0,6.0,6.0,9.0,6.0,29.0,15.0,0.0,8.0,6.0,1.0,1.0,0.0,0.0
1,1.0,2.0,7.0,12.0,12.0,11.0,7.0,7.0,8.0,8.0,3.0,17.0,7.0,0.0,6.0,7.0,3.0,0.0,0.0,0.0
2,1.0,3.0,1.0,11.0,4.0,3.0,8.0,10.0,3.0,8.0,0.0,22.0,17.0,3.0,7.0,5.0,3.0,0.0,3.0,0.0
3,1.0,4.0,8.0,17.0,4.0,4.0,4.0,13.0,4.0,2.0,2.0,8.0,10.0,32.0,3.0,0.0,2.0,0.0,4.0,1.0
4,1.0,5.0,3.0,27.0,0.0,9.0,3.0,10.0,5.0,2.0,3.0,7.0,1.0,7.0,2.0,1.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,30.0,16.0,30.0,30.0,18.0,5.0,35.0,0.0,0.0,22.0,2.0,100.0,74.0,0.0,47.0,1.0,4.0,0.0,3.0,2.0
596,30.0,17.0,21.0,12.0,0.0,6.0,0.0,0.0,1.0,31.0,5.0,32.0,88.0,0.0,23.0,2.0,1.0,0.0,0.0,6.0
597,30.0,18.0,25.0,13.0,1.0,5.0,2.0,1.0,0.0,40.0,4.0,27.0,52.0,0.0,23.0,3.0,4.0,0.0,0.0,5.0
598,30.0,19.0,22.0,6.0,2.0,9.0,1.0,1.0,1.0,26.0,2.0,30.0,45.0,1.0,18.0,0.0,0.0,0.0,1.0,12.0


## FOV spot location

In [30]:
def get_spot_x_y_range(x_global_px, y_global_px, fov_id, fov_spot_coordinates):
    fov_length = 5472
    fov_width = 3648
    x_l = x_global_px
    y_l = y_global_px
    x_h = x_l + fov_length
    y_h = y_l + fov_width
    
    for i in range(1, 6):  # 5
        for j in range(1, 5):  # 4
            spot_id = (i - 1) * 4 + j
            x = x_l + (5472 / 10.0) * 1 + (5472 / 5.0) * (i - 1)
            y = y_h - (3648 / 8.0) * 1 - (3648 / 4.0) * (j - 1)
            fov_spot_coordinates = fov_spot_coordinates.append(
                {'fov': fov_id, 'spot_id': spot_id, 'x': x, 'y': y},
                ignore_index=True
            )

    fov_spot_coordinates['x'] = fov_spot_coordinates['x']
    fov_spot_coordinates['y'] = fov_spot_coordinates['y']

    # fov_spot_coordinates['x'] = fov_spot_coordinates['x'] * 0.18 * 1e-4
    # fov_spot_coordinates['y'] = fov_spot_coordinates['y'] * 0.18 * 1e-4

    return fov_spot_coordinates



In [31]:
fov_spot_coordinates = pd.DataFrame(columns = ['fov', 'spot_id', 'x', 'y'])
for fov_id in fov_lst:
    x_px = fov_position[fov_position['fov']==fov_id]["x_global_px"].values[0]
    y_px = fov_position[fov_position['fov']==fov_id]["y_global_px"].values[0]
    fov_spot_coordinates = get_spot_x_y_range(x_px, y_px, fov_id, fov_spot_coordinates)
fov_spot_coordinates

,fov,spot_id,x,y
0,1.0,1.0,3736.088889,158408.666667
1,1.0,2.0,3736.088889,157496.666667
2,1.0,3.0,3736.088889,156584.666667
3,1.0,4.0,3736.088889,155672.666667
4,1.0,5.0,4830.488889,158408.666667
...,...,...,...,...
595,30.0,16.0,28908.177778,173922.666667
596,30.0,17.0,30002.577778,176658.666667
597,30.0,18.0,30002.577778,175746.666667
598,30.0,19.0,30002.577778,174834.666667


## generate H5ad file

In [33]:
spot_gene_expression = spot_gene_expression.drop(columns = ["fov", "spot_id"])
ground_truth_table = ground_truth_table.drop(columns = ["fov", "spot_id"])
fov_spot_coordinates = fov_spot_coordinates.drop(columns = ["fov", "spot_id"])



import anndata as ad
st_adata = ad.AnnData(X = spot_gene_expression.values, obs = ground_truth_table, var = pd.DataFrame(index = list(spot_gene_expression.columns)), dtype=int)
st_adata.obsm["spatial"] = fov_spot_coordinates.values

spot_sums = np.sum(st_adata.X, axis=1)
mask = spot_sums > 100
filtered_data = st_adata[mask]

file_path = "/home/luqiaolin/projects/Benchmarking_paper_code/pseudo_spot_generation/cosmx_lung/Lung5_1.h5ad"
filtered_data.write_h5ad(file_path)

